In [5]:
import platform
platform.python_version()

'3.6.7'

In [6]:
#coding=utf-8
import numpy as np
import pickle
import os
import time
import sys
import shutil
import skimage 

In [13]:
#caffe_root = '/home/cscl/caffe-master/' 
#sys.path.insert(0, caffe_root + 'python')
import caffe
GPU_ID = 0 # Switch between 0 and 1 depending on the GPU you want to use.
caffe.set_mode_gpu()
caffe.set_device(GPU_ID)

net_file = 'googlenet_deploy.prototxt'
caffe_model = 'models/googlenet_hccr.caffemodel' 
mean_file = 'meanfiles/CASIA1.0_1.1_1.2_mean_112.npy'
unicode_index = np.loadtxt('util/unicode_index.txt', delimiter = ',',dtype = np.int) #7534
net = caffe.Net(net_file,caffe_model,caffe.TEST)

In [14]:
def get_crop_image(imagepath, img_name):
    img=skimage.io.imread(os.path.join(imagepath,img_name),as_grey=True)
    black_index = np.where(img < 255 )
    min_x = min(black_index[0])
    max_x = max(black_index[0])
    min_y = min(black_index[1])
    max_y = max(black_index[1])
    #print(min_x,max_x,min_y,max_y)
    image = caffe.io.load_image(os.path.join(imagepath,img_name))
    return image[min_x:max_x, min_y:max_y,:]

In [15]:
def evaluate_content(imagepath, top_k):
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_raw_scale('data', 255) 

    rightcount=0
    allcount=0

    allimage=os.listdir(imagepath)
    allimage = [img_name for img_name in allimage if 'fake_B' in img_name]

    for img_name in allimage:
        allcount = allcount + 1
        label_truth = img_name.split('.')[0].split('_fake_B')[0] # unicode of this character

        print ("----------------------")
        image = get_crop_image(imagepath,img_name)
        net.blobs['data'].data[...] = transformer.preprocess('data',image)
        out = net.forward()
        label_index = net.blobs['loss'].data[0].flatten().argsort()[-1:-top_k-1:-1]
        labels = unicode_index[label_index.astype(np.int)]  # output unicode

        #print 'Index: ',label_index 
        print ('Top-' + str(top_k) + ' Label: ',labels)
        print ('label_truth: ',label_truth)

        for i in range(0,top_k):
            if  labels[i] == int(label_truth):
                rightcount=rightcount+1
                break
    print('RESULT')
    print(rightcount,allcount,(float)(rightcount)/(float)(allcount))


In [16]:
def predict(path, top_k):
    '''
    predict top-k confident
    :param path: image path
    :param top_k:
    :return labels: top-k confident labels
    '''
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_raw_scale('data', 255) 
    
    imagepath, img_name = os.path.dirname(path), os.path.basename(path)
    image = get_crop_image(imagepath,img_name)
    
    net.blobs['data'].data[...] = transformer.preprocess('data',image)
    out = net.forward()
    label_index = net.blobs['loss'].data[0].flatten().argsort()[-1:-top_k-1:-1]
    labels = unicode_index[label_index.astype(np.int)]  # output unicode
    
    return labels

In [17]:
def get_layer(path, layer_name):
    '''
    get output of the layer
    '''
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_raw_scale('data', 255) 
    
    imagepath, img_name = os.path.dirname(path), os.path.basename(path)
    image = get_crop_image(imagepath,img_name)
    
    net.blobs['data'].data[...] = transformer.preprocess('data',image)
    out = net.forward()
    return net.blobs[layer_name].data[0]

# Convert unicode index to character

In [18]:
def unicode_to_char(unicode_index):
    return hex(unicode_index).replace('0x','\\u').encode().decode('unicode-escape')

In [40]:
unicode_to_char(19978)

'上'

# Convert character to unicode index

In [19]:
def char_to_unicode(char):
    return int(char.encode('unicode-escape').decode().replace('\\u', '0x'), 16)

In [12]:
char_to_unicode('层')

23618

In [84]:
if __name__=='__main__':
    imagepath='images/'
    top_k = 1;
    # evaluate(imagepath,top_k)
    print (unicode_to_char(predict('../input/simhei_to_1252/trainA/21187.png',top_k)[0]))

勃


In [14]:
net.blobs

OrderedDict([('data', <caffe._caffe.Blob at 0x7ffa7439f2f0>),
             ('conv1', <caffe._caffe.Blob at 0x7ffa7439f348>),
             ('pool1', <caffe._caffe.Blob at 0x7ffa7439f298>),
             ('norm1', <caffe._caffe.Blob at 0x7ffa7439f3a0>),
             ('reduction2', <caffe._caffe.Blob at 0x7ffa7439f3f8>),
             ('conv2', <caffe._caffe.Blob at 0x7ffa7439f450>),
             ('norm2', <caffe._caffe.Blob at 0x7ffa7439f4a8>),
             ('pool2', <caffe._caffe.Blob at 0x7ffa7439f500>),
             ('pool2_pool2_0_split_0', <caffe._caffe.Blob at 0x7ffa7439f558>),
             ('pool2_pool2_0_split_1', <caffe._caffe.Blob at 0x7ffa7439f5b0>),
             ('pool2_pool2_0_split_2', <caffe._caffe.Blob at 0x7ffa7439f608>),
             ('pool2_pool2_0_split_3', <caffe._caffe.Blob at 0x7ffa7439f660>),
             ('icp1_reduction1', <caffe._caffe.Blob at 0x7ffa7439f6b8>),
             ('icp1_reduction2', <caffe._caffe.Blob at 0x7ffa7439f710>),
             ('icp1_pool', <ca

In [105]:
get_layer('images/26049.bmp', 'icp3_in').shape

(480, 7, 7)

In [106]:
get_layer('images/26049.bmp', 'icp2_out').shape

(480, 14, 14)

In [107]:
get_layer('images/26049.bmp', 'icp3_in_icp3_in_0_split_0').shape

(480, 7, 7)

In [108]:
get_layer('images/26049.bmp', 'icp3_in_icp3_in_0_split_1').shape

(480, 7, 7)

In [20]:
def gram_matrix(feat):
    # C x H x W
    C,H,W = feat.shape
    feat = feat.reshape((C,-1))
    gram = feat.dot(feat.transpose())/(C**2)
    return gram

In [62]:
gram = gram_matrix(get_layer('images/26049.bmp', 'icp2_out')) # out put of icp2 is the input of icp3

In [63]:
gram.shape

(480, 480)

In [22]:
def style_rep(path, layer):
    ''' Style representation of the image
    :param path: input image path
    :layer: layer l where we calculate the gram matrix
    :return style_rep: style representation in np array
    '''
    icp3_in = get_layer(path, layer)
    return gram_matrix(icp3_in)

In [67]:
np.linalg.norm(style_rep('images/26049.bmp')-style_rep('../input/simhei_to_1252/trainA'))

TypeError: style_rep() missing 1 required positional argument: 'layer'

In [152]:
np.linalg.norm(style_rep('images/26049.bmp')-style_rep('images/26257.bmp'))

66335048.0

In [153]:
np.linalg.norm(style_rep('images/26049.bmp')-style_rep('images/26263.bmp'))

64735112.0

In [35]:
np.linalg.norm(style_rep('../input/simhei_to_1252/trainA/19981.png')-style_rep('../input/simhei_to_1252/trainA/20135.png'))

45563864.0

In [66]:
style_rep('../input/simhei_to_1252/trainA/19981.png', 'icp3_in')

array([[ 0.50044328,  0.08398292,  0.25962034, ...,  0.28180874,
         0.02621776,  0.69787326],
       [ 0.08398292,  0.17111357,  0.18687024, ...,  0.48882158,
         0.01568675,  0.86435852],
       [ 0.25962034,  0.18687024,  0.81742588, ...,  0.8821717 ,
         0.05669476,  1.70574368],
       ..., 
       [ 0.28180874,  0.48882158,  0.8821717 , ...,  2.06422879,
         0.08021436,  3.61852159],
       [ 0.02621776,  0.01568675,  0.05669476, ...,  0.08021436,
         0.00898808,  0.14064938],
       [ 0.69787326,  0.86435852,  1.70574368, ...,  3.61852159,
         0.14064938,  6.82266547]])

In [68]:
style_rep('../input/simhei_to_1252/trainA/19993.png', 'icp3_in')

array([[ 0.29038055,  0.09918299,  0.12560867, ...,  0.14959786,
         0.01184188,  0.55951026],
       [ 0.09918299,  0.24877704,  0.19264106, ...,  0.42872033,
         0.01842652,  0.77314819],
       [ 0.12560867,  0.19264106,  0.94178216, ...,  0.57414537,
         0.05455829,  1.64499919],
       ..., 
       [ 0.14959786,  0.42872033,  0.57414537, ...,  1.48292589,
         0.04617327,  2.38988091],
       [ 0.01184188,  0.01842652,  0.05455829, ...,  0.04617327,
         0.00574668,  0.11711765],
       [ 0.55951026,  0.77314819,  1.64499919, ...,  2.38988091,
         0.11711765,  5.24544108]])

# Average root-mean-square-different between style representations of images in A

In [16]:
res = 0.0
a_dir = '../datasets/simhei_to_1252/trainA'
a_dir_1 = '../datasets/simhei_to_1252/testA'
layer = 'icp3_in'
image_names = [os.path.join(a_dir,e) for e in os.listdir(a_dir)] + [os.path.join(a_dir_1, e) for e in os.listdir(a_dir_1)]
np.random.shuffle(image_names)
n = len(image_names)
n = n if n%2==0 else n-1
for i in range(0,n,2):
    res += np.linalg.norm(style_rep(image_names[i],layer)-style_rep(image_names[i+1],layer))
res /= n/2.0
print (res)

418.733214524


# Average rmsd between style representations of images in A and B

In [17]:
res = 0.0
a_dir = '../datasets/simhei_to_1252/trainA'
a_dir_1 = '../datasets/simhei_to_1252/testA'
b_dir = '../datasets/simhei_to_1252/trainB'
b_dir_1 = '../datasets/simhei_to_1252/testB'
layer = 'icp3_in'
image_a_names = [os.path.join(a_dir, e) for e in os.listdir(a_dir)] + [os.path.join(a_dir_1, e) for e in os.listdir(a_dir_1)]
np.random.shuffle(image_a_names)
image_b_names = [os.path.join(b_dir, e) for e in os.listdir(b_dir)] + [os.path.join(b_dir_1, e) for e in os.listdir(b_dir_1)]
np.random.shuffle(image_b_names)
n = min(len(image_a_names), len(image_b_names))
n = n if n%2==0 else n-1
for i in range(0,n,1):
    res += np.linalg.norm(style_rep(image_a_names[i],layer)-style_rep(image_b_names[i],layer))
res /= n
print (res)

555.923504489


# Average rmsd between style representations of images in B

In [ ]:
# res = 0.0
# a_dir = '../input/simhei_to_1252/trainB'
# layer = 'icp3_in'
# image_names = os.listdir(a_dir)
# np.random.shuffle(image_names)
# n = len(image_names)
# n = n if n%2==0 else n-1
# for i in range(0,n,2):
#     res += np.linalg.norm(style_rep(os.path.join(a_dir, image_names[i]),layer)-style_rep(os.path.join(a_dir, image_names[i+1]),layer))
# res /= n/2
# print (res)

# Evaluate top-5 accuracy in the generated images

In [19]:
evaluate_content('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_1_simhei_to_1252/test_latest/images',5)
evaluate_content('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_2_simhei_to_1252/test_latest/images',5)

----------------------
Top-5 Label:  [20108  8592 20128    61  8594]
label_truth:  19968
----------------------
Top-5 Label:  [19969 21378    84 19979    74]
label_truth:  19969
----------------------
Top-5 Label:  [19971 22763   116 21313 21269]
label_truth:  19971
----------------------
Top-5 Label:  [21381 26041 19975 24207 21385]
label_truth:  19975
----------------------
Top-5 Label:  [19976 22823 26408 22899 22763]
label_truth:  19976
----------------------
Top-5 Label:  [19977    69 27491 20129  8712]
label_truth:  19977
----------------------
Top-5 Label:  [19978 27490 22303  8869 19992]
label_truth:  19978
----------------------
Top-5 Label:  [19982 25908 21005 21248 21344]
label_truth:  19982
----------------------
Top-5 Label:  [19987 40614 32771 23389 20940]
label_truth:  19987
----------------------
Top-5 Label:  [19988 30446 26086 26085 26352]
label_truth:  19988
----------------------
Top-5 Label:  [19990 22320 20063 22336 27844]
label_truth:  19990
---------------------

Top-5 Label:  [20182 20311 22320 20185 20504]
label_truth:  20182
----------------------
Top-5 Label:  [20183 20181 20238 20202 20184]
label_truth:  20183
----------------------
Top-5 Label:  [20160 21319    72 24318 33401]
label_truth:  20184
----------------------
Top-5 Label:  [20185 35753 35754 20284 21152]
label_truth:  20185
----------------------
Top-5 Label:  [20196 20170 21547 25343 20840]
label_truth:  20196
----------------------
Top-5 Label:  [20197 21485 23665 23746 27490]
label_truth:  20197
----------------------
Top-5 Label:  [20202 20249 20284 35758 25912]
label_truth:  20202
----------------------
Top-5 Label:  [33313 21360 30340 32418 26020]
label_truth:  20204
----------------------
Top-5 Label:  [20208 24443 20570 20211 20539]
label_truth:  20208
----------------------
Top-5 Label:  [20210 20214 20182 20280 20167]
label_truth:  20210
----------------------
Top-5 Label:  [20214 20210 20367 20213 20191]
label_truth:  20214
----------------------
Top-5 Label:  [20215 

Top-5 Label:  [20613 20754 20723 20539 20616]
label_truth:  20613
----------------------
Top-5 Label:  [20616 20723 20613 20456 20539]
label_truth:  20616
----------------------
Top-5 Label:  [20493 20621 20444 20572 20449]
label_truth:  20621
----------------------
Top-5 Label:  [20643 20472 20518 20469 20454]
label_truth:  20643
----------------------
Top-5 Label:  [20648 20731 20652 20332 20556]
label_truth:  20648
----------------------
Top-5 Label:  [20652 20474 20718 20698 20540]
label_truth:  20652
----------------------
Top-5 Label:  [20667 20426 20679 24930 20462]
label_truth:  20667
----------------------
Top-5 Label:  [20687 20360 20462 20621 20711]
label_truth:  20687
----------------------
Top-5 Label:  [20698 20462 20666 20717 32557]
label_truth:  20698
----------------------
Top-5 Label:  [20711 20616 20725 20754 20517]
label_truth:  20711
----------------------
Top-5 Label:  [20616 20723 20698 20613 20687]
label_truth:  20723
----------------------
Top-5 Label:  [20652 

Top-5 Label:  [21002 25170 37015 25187 21010]
label_truth:  21002
----------------------
Top-5 Label:  [21009 37026 24418 33606 21002]
label_truth:  21009
----------------------
Top-5 Label:  [21010 22914 22937 22902 21016]
label_truth:  21010
----------------------
Top-5 Label:  [21015 27522 27521 39535 28872]
label_truth:  21015
----------------------
Top-5 Label:  [21016 21051 21058 34917 37066]
label_truth:  21016
----------------------
Top-5 Label:  [21017 21006 21539 21018 27979]
label_truth:  21017
----------------------
Top-5 Label:  [21018 21047 29935 37044 21397]
label_truth:  21018
----------------------
Top-5 Label:  [21019 21032 25111 21073  9318]
label_truth:  21019
----------------------
Top-5 Label:  [28078 29642 21024 21009 30178]
label_truth:  21024
----------------------
Top-5 Label:  [21028 21010 37030 21066 25187]
label_truth:  21028
----------------------
Top-5 Label:  [21032 21019 24840 21253 40101]
label_truth:  21032
----------------------
Top-5 Label:  [21033 

Top-5 Label:  [   53   114 21378 24191 30098]
label_truth:  21378
----------------------
Top-5 Label:  [21380 21358 21387 24065 21382]
label_truth:  21380
----------------------
Top-5 Label:  [21382 21385 19993 21381 19975]
label_truth:  21382
----------------------
Top-5 Label:  [21385 21381 21518 21358 21382]
label_truth:  21385
----------------------
Top-5 Label:  [20219 27754 36801 21387 22313]
label_truth:  21387
----------------------
Top-5 Label:  [21388 21387 30690 27663 20164]
label_truth:  21388
----------------------
Top-5 Label:  [21397 20458 20029 37246 21410]
label_truth:  21397
----------------------
Top-5 Label:  [21400 21283 38634 28270 24283]
label_truth:  21400
----------------------
Top-5 Label:  [28525 28182 28319 28316 24822]
label_truth:  21402
----------------------
Top-5 Label:  [21407 21402 21400 21283 24243]
label_truth:  21407
----------------------
Top-5 Label:  [21410 38593 23524 35070 38191]
label_truth:  21410
----------------------
Top-5 Label:  [20667 

...

Top-5 Label:  [36420 36479 21884 36387 36393]
label_truth:  36420
----------------------
Top-5 Label:  [36424 36362 23743 40843 40836]
label_truth:  36424
----------------------
Top-5 Label:  [36427 36386 36479 36432 36461]
label_truth:  36427
----------------------
Top-5 Label:  [37246 36495 28698 36902 36454]
label_truth:  36454
----------------------
Top-5 Label:  [36460 23965 36367 36510 30634]
label_truth:  36460
----------------------
Top-5 Label:  [36477 36466 36479 36510 36470]
label_truth:  36466
----------------------
Top-5 Label:  [36479 31726 36398 36468 36444]
label_truth:  36479
----------------------
Top-5 Label:  [36393 36318 36481 36470 36468]
label_truth:  36481
----------------------
Top-5 Label:  [36425 36463 32592 36476 36500]
label_truth:  36487
----------------------
Top-5 Label:  [36523 27670 35282 21331 25149]
label_truth:  36523
----------------------
Top-5 Label:  [36524 35970 40501 33285 37036]
label_truth:  36524
----------------------
Top-5 Label:  [36527 

Top-5 Label:  [36930 36880 38567 21828 35836]
label_truth:  36930
----------------------
Top-5 Label:  [36935 36973 21294 36875 36952]
label_truth:  36935
----------------------
Top-5 Label:  [36941 29048 21310 36417 35082]
label_truth:  36941
----------------------
Top-5 Label:  [36943 36935 36386 36962 24858]
label_truth:  36943
----------------------
Top-5 Label:  [21992 21870 21622 21638 38519]
label_truth:  36947
----------------------
Top-5 Label:  [29830 25788 29794 22055 25022]
label_truth:  36951
----------------------
Top-5 Label:  [36951 35892 36963 21294 32561]
label_truth:  36963
----------------------
Top-5 Label:  [22124 21822 30561 36965 26214]
label_truth:  36965
----------------------
Top-5 Label:  [36973 36952 36981 36875 36974]
label_truth:  36973
----------------------
Top-5 Label:  [36991 36965 36842 36923 36894]
label_truth:  36981
----------------------
Top-5 Label:  [36992 28608 36968 28196 28510]
label_truth:  36992
----------------------
Top-5 Label:  [37011 

Top-5 Label:  [38060 38091 38037 38054 38135]
label_truth:  38128
----------------------
Top-5 Label:  [38129 38128 38157 38102 39291]
label_truth:  38129
----------------------
Top-5 Label:  [38130 33298 36723 38126 38189]
label_truth:  38130
----------------------
Top-5 Label:  [38162 38134 38189 38181 29701]
label_truth:  38134
----------------------
Top-5 Label:  [38136 38166 38096 38210 38167]
label_truth:  38136
----------------------
Top-5 Label:  [38225 38256 38241 38242 38217]
label_truth:  38138
----------------------
Top-5 Label:  [38142 40098 28063 38069 38181]
label_truth:  38142
----------------------
Top-5 Label:  [38144 38808 38145 25422 34552]
label_truth:  38144
----------------------
Top-5 Label:  [32517 38148 38234 40748 40147]
label_truth:  38148
----------------------
Top-5 Label:  [38149 38110 38177 34583 22490]
label_truth:  38149
----------------------
Top-5 Label:  [38152 32483 29703 38240 38041]
label_truth:  38152
----------------------
Top-5 Label:  [38155 

Top-5 Label:  [38533 36367 39576 36413 38470]
label_truth:  38533
----------------------
Top-5 Label:  [38534 38477 38541 32903 38519]
label_truth:  38534
----------------------
Top-5 Label:  [38539 38534 38556 38517 38519]
label_truth:  38539
----------------------
Top-5 Label:  [38543 38519 38567 38475 38505]
label_truth:  38543
----------------------
Top-5 Label:  [38534 38505 38544 38519 38506]
label_truth:  38544
----------------------
Top-5 Label:  [38548 38539 38476 23646 23641]
label_truth:  38548
----------------------
Top-5 Label:  [27463 40515 38553 39627 30805]
label_truth:  38553
----------------------
Top-5 Label:  [38514 38556 38548 38553 38480]
label_truth:  38556
----------------------
Top-5 Label:  [38567 38552 39130 37274 30896]
label_truth:  38567
----------------------
Top-5 Label:  [38582 22839 31177 23609 32793]
label_truth:  38582
----------------------
Top-5 Label:  [20339 23828 38585 20652 31568]
label_truth:  38592
----------------------
Top-5 Label:  [38593 

Top-5 Label:  [39184 30342 26234 30498 31570]
label_truth:  39184
----------------------
Top-5 Label:  [39269 35749 32418 21368 21164]
label_truth:  39269
----------------------
Top-5 Label:  [39277 39278 39042 38203 39041]
label_truth:  39277
----------------------
Top-5 Label:  [39278 38054 39038 25947 27431]
label_truth:  39278
----------------------
Top-5 Label:  [39279 39286 38065 39272 25111]
label_truth:  39279
----------------------
Top-5 Label:  [39281 39287 39301 39311 40077]
label_truth:  39281
----------------------
Top-5 Label:  [39286 32469 20389 23046 38105]
label_truth:  39286
----------------------
Top-5 Label:  [39291 39290 28798 23443 40091]
label_truth:  39290
----------------------
Top-5 Label:  [39292 39295 39285 35299 39279]
label_truth:  39292
----------------------
Top-5 Label:  [20469 39295 20420 20470 20551]
label_truth:  39295
----------------------
Top-5 Label:  [39297 39295 39275 39274 38040]
label_truth:  39297
----------------------
Top-5 Label:  [39301 

top-5 content accuracy  
**Train ratio: 0.1, RESNET-6: 86.53%**  
**Train ratio: 0.1, DENSENET-5: 82.63%**  

# Define function for evaluating style

In [16]:
def evaluate_style(imagepath):
    layer = 'icp3_in'
    
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_raw_scale('data', 255) 

    rightcount=0
    allcount=0

    gen_images = os.listdir(imagepath)
    gen_images = [img_name for img_name in gen_images if '_fake_B' in img_name]
    tar_images = os.listdir('../datasets/simhei_to_1252/testA')
    
    res = 0.0
    n = min(len(gen_images), len(tar_images))
    n = n if n%2==0 else n-1
    for i in range(0,n,1):
        res += np.linalg.norm(style_rep(os.path.join(imagepath, gen_images[i]),layer)-style_rep(os.path.join('../datasets/simhei_to_1252/testA/', tar_images[i]),layer))
    res /= n
    
    print ('Average style discrepancy %f'%res)
    

# Average style discrepancy between generated images and ground truth target images

In [29]:
evaluate_style('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_1_simhei_to_1252/test_latest/images')
evaluate_style('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_2_simhei_to_1252/test_latest/images')

Average style discrepancy 384.215083
Average style discrepancy 381.934116


# Evaluate top-5 accuracy in the generated images

In [ ]:
evaluate_content('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_4_simhei_to_1252/test_latest/images',5)

top-5 content accuracy  
**Train ratio: 0.15, RESNET-6: 83.13%**

In [13]:
evaluate_content('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_5_simhei_to_1252/test_latest/images',5)

----------------------
Top-5 Label:  [  247 23424 20013  8592 20128]
label_truth:  19968
----------------------
Top-5 Label:  [20010 19979 21315 21313 24178]
label_truth:  19969
----------------------
Top-5 Label:  [19971 20129 22763   116 24331]
label_truth:  19971
----------------------
Top-5 Label:  [29255 24207 21381 24038 26020]
label_truth:  19975
----------------------
Top-5 Label:  [19976 22899 22303 22763 25903]
label_truth:  19976
----------------------
Top-5 Label:  [19977    69 20129 20043  8780]
label_truth:  19977
----------------------
Top-5 Label:  [19978 21340 24516    53  8595]
label_truth:  19978
----------------------
Top-5 Label:  [19982 21344 13199 19978 21320]
label_truth:  19982
----------------------
Top-5 Label:  [19987 40614 21578 24403 32771]
label_truth:  19987
----------------------
Top-5 Label:  [19988 36807 33251 32819 21281]
label_truth:  19988
----------------------
Top-5 Label:  [19990 24319 29976 22336 19994]
label_truth:  19990
---------------------

Top-5 Label:  [20249 20284   117 25912 20203]
label_truth:  20249
----------------------
Top-5 Label:  [20254 27709 27718 20177 31860]
label_truth:  20254
----------------------
Top-5 Label:  [20255 20276 32428 20315 20307]
label_truth:  20255
----------------------
Top-5 Label:  [20256 20238 20342 20272 20351]
label_truth:  20256
----------------------
Top-5 Label:  [20260 20221 20223 20215 20320]
label_truth:  20260
----------------------
Top-5 Label:  [20262 20316 20263 32438 20461]
label_truth:  20262
----------------------
Top-5 Label:  [20266 22955 27817 21584 20162]
label_truth:  20266
----------------------
Top-5 Label:  [20271 20336 20267 24597 20291]
label_truth:  20271
----------------------
Top-5 Label:  [20272 20181 20342 20237 20305]
label_truth:  20272
----------------------
Top-5 Label:  [20276 20335 32458 25292 20603]
label_truth:  20276
----------------------
Top-5 Label:  [20278 24604 20393 20461 20221]
label_truth:  20278
----------------------
Top-5 Label:  [20280 

Top-5 Label:  [33216 32755 29237 26013 39663]
label_truth:  20864
----------------------
Top-5 Label:  [20869 32905 38378 28796 27728]
label_truth:  20869
----------------------
Top-5 Label:  [20873 30003 30001 20877 26354]
label_truth:  20873
----------------------
Top-5 Label:  [20876 29642 37026 33337 26379]
label_truth:  20876
----------------------
Top-5 Label:  [20877 20873 38632 25215 37193]
label_truth:  20877
----------------------
Top-5 Label:  [20540 23452 31530 30041 20356]
label_truth:  20882
----------------------
Top-5 Label:  [23916 35273 31650 30097 24005]
label_truth:  20885
----------------------
Top-5 Label:  [20887 21650 20801 20804 37095]
label_truth:  20887
----------------------
Top-5 Label:  [21326 20973 20342 31481 36161]
label_truth:  20889
----------------------
Top-5 Label:  [28814 20892 35772 35848 20852]
label_truth:  20892
----------------------
Top-5 Label:  [20896 36345 36801 36214 32568]
label_truth:  20896
----------------------
Top-5 Label:  [23916 

Top-5 Label:  [21271 22766 25199 25288 25169]
label_truth:  21271
----------------------
Top-5 Label:  [21273 39064 36286 40668 24433]
label_truth:  21273
----------------------
Top-5 Label:  [21277 20542 20208 21276 36814]
label_truth:  21277
----------------------
Top-5 Label:  [21281 24311 36802 36866 24040]
label_truth:  21281
----------------------
Top-5 Label:  [23648 35834 20556 28009 21311]
label_truth:  21311
----------------------
Top-5 Label:  [21313 21345 21342   102 22763]
label_truth:  21313
----------------------
Top-5 Label:  [24178 25164 23544 21313 20110]
label_truth:  21315
----------------------
Top-5 Label:  [20117 24320 20025 23609 21319]
label_truth:  21319
----------------------
Top-5 Label:  [29275 21320 24180 25163 22833]
label_truth:  21320
----------------------
Top-5 Label:  [21321 26412 25102 20117 25190]
label_truth:  21321
----------------------
Top-5 Label:  [21322 31859 24403    42 29275]
label_truth:  21322
----------------------
Top-5 Label:  [21327 

Top-5 Label:  [26133 26121 21548 30024 30010]
label_truth:  21548
----------------------
Top-5 Label:  [30460 26103 26126 26121 30504]
label_truth:  21550
----------------------
Top-5 Label:  [32930 26106 21553 30024 30504]
label_truth:  21553
----------------------
Top-5 Label:  [28805 33278 31505 30690 21471]
label_truth:  21556
----------------------
Top-5 Label:  [21557 30471 21499 26103 21535]
label_truth:  21557
----------------------
Top-5 Label:  [39034 21560 26247 21591 39031]
label_truth:  21560
----------------------
Top-5 Label:  [21561 21591 21550 21545 22350]
label_truth:  21561
----------------------
Top-5 Label:  [21563 21561 21500 21480 21545]
label_truth:  21563
----------------------
Top-5 Label:  [21564 21500 21501 22235 35759]
label_truth:  21564
----------------------
Top-5 Label:  [21566 21542 33292 33493 21484]
label_truth:  21566
----------------------
Top-5 Label:  [21568 21590 21548 21505 21621]
label_truth:  21568
----------------------
Top-5 Label:  [21574 

...

Top-5 Label:  [36776 23241 22539 38604 23072]
label_truth:  36776
----------------------
Top-5 Label:  [31255 31204 37026 31234 31245]
label_truth:  36777
----------------------
Top-5 Label:  [24199 23617 29256 20247 25104]
label_truth:  36784
----------------------
Top-5 Label:  [31578 39286 38243 27032 24863]
label_truth:  36785
----------------------
Top-5 Label:  [36793 20063 21151 27744 20116]
label_truth:  36793
----------------------
Top-5 Label:  [36797 35746 35760 36802 36214]
label_truth:  36797
----------------------
Top-5 Label:  [36798 35753 27760 22836 35776]
label_truth:  36798
----------------------
Top-5 Label:  [36802 36801 36807 24311 27754]
label_truth:  36802
----------------------
Top-5 Label:  [36804 36884 20461 36840 20278]
label_truth:  36804
----------------------
Top-5 Label:  [36805 35759 36827 27739 36793]
label_truth:  36805
----------------------
Top-5 Label:  [36807 35752 31038 36801 36802]
label_truth:  36807
----------------------
Top-5 Label:  [36808 

Top-5 Label:  [37275 37257 37237 37241 37245]
label_truth:  37237
----------------------
Top-5 Label:  [37238 37197 37232 37210 37266]
label_truth:  37238
----------------------
Top-5 Label:  [37213 37218 37197 37266 37255]
label_truth:  37239
----------------------
Top-5 Label:  [37197 37213 37210 37245 37238]
label_truth:  37240
----------------------
Top-5 Label:  [37255 37245 40560 37257 37253]
label_truth:  37255
----------------------
Top-5 Label:  [37207 37213 37214 21290 37217]
label_truth:  37257
----------------------
Top-5 Label:  [37239 37220 37259 37245 37237]
label_truth:  37259
----------------------
Top-5 Label:  [37290 37239 37300 37226 37228]
label_truth:  37266
----------------------
Top-5 Label:  [37214 37198 37220 37274 37219]
label_truth:  37274
----------------------
Top-5 Label:  [40560 37239 40637 37226 24223]
label_truth:  37275
----------------------
Top-5 Label:  [37319 24694 32600 33476 31859]
label_truth:  37319
----------------------
Top-5 Label:  [31243 

Top-5 Label:  [38378 27720 38377 19993 20869]
label_truth:  38378
----------------------
Top-5 Label:  [38381 38390 38386 38389 38393]
label_truth:  38381
----------------------
Top-5 Label:  [38382 38388 27827 38391 27753]
label_truth:  38382
----------------------
Top-5 Label:  [38383 38397 38420 38398 38382]
label_truth:  38383
----------------------
Top-5 Label:  [38384 38394 22237 38379 30000]
label_truth:  38384
----------------------
Top-5 Label:  [38388 38395 38403 38397 38384]
label_truth:  38388
----------------------
Top-5 Label:  [38391 38382 38378 38388 27827]
label_truth:  38391
----------------------
Top-5 Label:  [38393 38385 38417 38386 38420]
label_truth:  38393
----------------------
Top-5 Label:  [38394 38385 38384 27941 22260]
label_truth:  38394
----------------------
Top-5 Label:  [38395 22275 38423 21416 38413]
label_truth:  38395
----------------------
Top-5 Label:  [38397 22244 31162 38384 38417]
label_truth:  38397
----------------------
Top-5 Label:  [38400 

Top-5 Label:  [39034 24697 20542 22104 23835]
label_truth:  39034
----------------------
Top-5 Label:  [39035 39031 39033 28066 27976]
label_truth:  39035
----------------------
Top-5 Label:  [39033 39037 39030 30805 39040]
label_truth:  39037
----------------------
Top-5 Label:  [39038 20542 39031 39042 39034]
label_truth:  39038
----------------------
Top-5 Label:  [39039 39036 39050 35275 39032]
label_truth:  39039
----------------------
Top-5 Label:  [39031 39033 28066 39036 34429]
label_truth:  39042
----------------------
Top-5 Label:  [39044 28394 39048 39032 35947]
label_truth:  39044
----------------------
Top-5 Label:  [39045 39047 39031 39057 38504]
label_truth:  39045
----------------------
Top-5 Label:  [39046 39052 39041 39048 39060]
label_truth:  39046
----------------------
Top-5 Label:  [39047 39056 39038 39065 21413]
label_truth:  39047
----------------------
Top-5 Label:  [39048 24047 21170 39046 39056]
label_truth:  39048
----------------------
Top-5 Label:  [39050 

Top-5 Label:  [39336 40150 37233 39612 40836]
label_truth:  40150
----------------------
Top-5 Label:  [33181 37274 37295 39621 37282]
label_truth:  40158
----------------------
Top-5 Label:  [20984 13199 39647 40485 20985]
label_truth:  40479
----------------------
Top-5 Label:  [29623 29756 40481 21149 29595]
label_truth:  40481
----------------------
Top-5 Label:  [21596 26214 21527 26112 40483]
label_truth:  40483
----------------------
Top-5 Label:  [40485 36142 27431 40481 27572]
label_truth:  40485
----------------------
Top-5 Label:  [40486 30684 30683 24418 24425]
label_truth:  40486
----------------------
Top-5 Label:  [26029 24418 40497 30027 40519]
label_truth:  40493
----------------------
Top-5 Label:  [31373 40548 32958 36335 38243]
label_truth:  40495
----------------------
Top-5 Label:  [40499 24616 38041 38150 29066]
label_truth:  40499
----------------------
Top-5 Label:  [40501 39548 34503 31381 37217]
label_truth:  40501
----------------------
Top-5 Label:  [40509 

top-5 content accuracy  
**Train ratio: 0.15, DENSENET-5: 76.24%**

In [17]:
evaluate_style('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_4_simhei_to_1252/test_latest/images')
evaluate_style('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_5_simhei_to_1252/test_latest/images')

Average style discrepancy 384.540128
Average style discrepancy 381.076276


# ETL

In [10]:
def is_chinese(char):
    return u'\u4e00' <= char <= u'\u9fff'

In [11]:
is_chinese('よ')

False

In [12]:
is_chinese('勃')

True

In [25]:
def evaluate_content(imagepath, top_k):
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_raw_scale('data', 255) 

    rightcount=0
    allcount=0

    allimage=os.listdir(imagepath)
    allimage = [img_name for img_name in allimage if 'fake_B' in img_name]

    for img_name in allimage:
        label_truth = img_name.split('.')[0].split('_fake_B')[0] # unicode of this character
        
        truth_char = unicode_to_char(int(label_truth))
        if not is_chinese(truth_char):
            continue
        
        allcount = allcount + 1
        print ("----------------------")
        image = get_crop_image(imagepath,img_name)
        net.blobs['data'].data[...] = transformer.preprocess('data',image)
        out = net.forward()
        label_index = net.blobs['loss'].data[0].flatten().argsort()[-1:-top_k-1:-1]
        labels = unicode_index[label_index.astype(np.int)]  # output unicode

        #print 'Index: ',label_index 
        print ('Top-' + str(top_k) + ' Label: ',labels)
        print ('label_truth: ',label_truth)

        for i in range(0,top_k):
            if  labels[i] == int(label_truth):
                rightcount=rightcount+1
                break
    print('RESULT')
    print(rightcount,allcount,(float)(rightcount)/(float)(allcount))


In [27]:
evaluate_content('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_6_simhei_to_1252/test_latest/images',5)

----------------------
Top-5 Label:  [31035 19981 21342 26415 31034]
label_truth:  19981
----------------------
Top-5 Label:  [20197 39278 38378 35782 25919]
label_truth:  20197
----------------------
Top-5 Label:  [20381 33300 20469 30913 31108]
label_truth:  20381
----------------------
Top-5 Label:  [40147 20539 31776 31740 34914]
label_truth:  20685
----------------------
Top-5 Label:  [20877 33490 30011 20873 38632]
label_truth:  20877
----------------------
Top-5 Label:  [20128 20108   247 19968  8592]
label_truth:  19968
----------------------
Top-5 Label:  [20109 24191 20110 23425 20010]
label_truth:  19969
----------------------
Top-5 Label:  [20039 20129 19971 22763 23376]
label_truth:  19971
----------------------
Top-5 Label:  [ 8801 19977 27701  8721 20129]
label_truth:  19977
----------------------
Top-5 Label:  [19978 19992 27490 27491 24037]
label_truth:  19978
----------------------
Top-5 Label:  [24310 36874 20985 19990 38491]
label_truth:  19990
---------------------

Top-5 Label:  [22352 28192 27605 31487 33416]
label_truth:  21322
----------------------
Top-5 Label:  [25638 29053 27204 25479 24936]
label_truth:  21332
----------------------
Top-5 Label:  [21335 21830 23703 29301 23578]
label_truth:  21335
----------------------
Top-5 Label:  [24906 21338 25077 24999 24910]
label_truth:  21338
----------------------
Top-5 Label:  [21360 33562 25152 37070 30340]
label_truth:  21360
----------------------
Top-5 Label:  [21407 24895 28304 28418 21414]
label_truth:  21407
----------------------
Top-5 Label:  [24005 28751 39608 39600 39628]
label_truth:  21427
----------------------
Top-5 Label:  [20002 21435 29577 20027 29572]
label_truth:  21435
----------------------
Top-5 Label:  [21385 24203 21453 31034 21518]
label_truth:  21453
----------------------
Top-5 Label:  [25910 23696 39031 21571 23679]
label_truth:  21454
----------------------
Top-5 Label:  [21462 32821 32844 32822 21161]
label_truth:  21462
----------------------
Top-5 Label:  [21464 

Top-5 Label:  [23637 23480 21417 40575 23625]
label_truth:  23637
----------------------
Top-5 Label:  [23646 23409 28431 23626 31162]
label_truth:  23646
----------------------
Top-5 Label:  [23721 30758 22120 20821 30342]
label_truth:  23721
----------------------
Top-5 Label:  [23736 31568 26013 31534 23815]
label_truth:  23736
----------------------
Top-5 Label:  [24703 27698 40763 24748 29237]
label_truth:  23798
----------------------
Top-5 Label:  [20204   109 20215 20208 24029]
label_truth:  24029
----------------------
Top-5 Label:  [24030 24070 24571 24521 24683]
label_truth:  24030
----------------------
Top-5 Label:  [29579 24037 20110 29577 20129]
label_truth:  24037
----------------------
Top-5 Label:  [21534 24038 24196 22307 33292]
label_truth:  24038
----------------------
Top-5 Label:  [24066 38886 31034 33470 31168]
label_truth:  24066
----------------------
Top-5 Label:  [24076 36196 32773 32948 25995]
label_truth:  24067
----------------------
Top-5 Label:  [21147 

Top-5 Label:  [33407 33459 33465 31496 23736]
label_truth:  26041
----------------------
Top-5 Label:  [26053 26062 26059 26052 26063]
label_truth:  26053
----------------------
Top-5 Label:  [26063 26052 26053 26044 26059]
label_truth:  26063
----------------------
Top-5 Label:  [26085 26352 30333 21475 22231]
label_truth:  26085
----------------------
Top-5 Label:  [26087 27753 27752 24402 27850]
label_truth:  26087
----------------------
Top-5 Label:  [26089 38428 21331 26097 30338]
label_truth:  26089
----------------------
Top-5 Label:  [26126 26379 26112 26102 26152]
label_truth:  26126
----------------------
Top-5 Label:  [26131 27675 33285 26095 40494]
label_truth:  26131
----------------------
Top-5 Label:  [26143 26482 26159 26197 20882]
label_truth:  26143
----------------------
Top-5 Label:  [33282 26149 30450 24847 28952]
label_truth:  26149
----------------------
Top-5 Label:  [26152 38460 37218 33332 37226]
label_truth:  26152
----------------------
Top-5 Label:  [26157 

Top-5 Label:  [28781 28779 35265 35272 28798]
label_truth:  28779
----------------------
Top-5 Label:  [29791 21741 29177 36331 32565]
label_truth:  28797
----------------------
Top-5 Label:  [22120 28845 24061 36160 26115]
label_truth:  28845
----------------------
Top-5 Label:  [29081 28982 29006 28893 29066]
label_truth:  28961
----------------------
Top-5 Label:  [28982 29022 28872 29087 24616]
label_truth:  28982
----------------------
Top-5 Label:  [24936 29791 36174 28903 36156]
label_truth:  28988
----------------------
Top-5 Label:  [29031 26157 29030 21823 29081]
label_truth:  29031
----------------------
Top-5 Label:  [29087 29100 35276 35279 20149]
label_truth:  29105
----------------------
Top-5 Label:  [29123 24936 25022 24980 38576]
label_truth:  29123
----------------------
Top-5 Label:  [34955 24517 21326 27606 36151]
label_truth:  29238
----------------------
Top-5 Label:  [29256 36317 39606 39610 39537]
label_truth:  29256
----------------------
Top-5 Label:  [21326 

Top-5 Label:  [32482 30388 27448 38601 32474]
label_truth:  32121
----------------------
Top-5 Label:  [29923 31252 24904 40118 23879]
label_truth:  32207
----------------------
Top-5 Label:  [30097 31411 31108 32511 33284]
label_truth:  32209
----------------------
Top-5 Label:  [40136 39055 32508 27032 31292]
label_truth:  32218
----------------------
Top-5 Label:  [35082 32534 29923 31411 36417]
label_truth:  32232
----------------------
Top-5 Label:  [39055 32517 32556 35082 29923]
label_truth:  32244
----------------------
Top-5 Label:  [31143 40777 25094 35157 38742]
label_truth:  32318
----------------------
Top-5 Label:  [31411 32545 40761 38260 40748]
label_truth:  32340
----------------------
Top-5 Label:  [32618 38671 37075 30386 32622]
label_truth:  32618
----------------------
Top-5 Label:  [32622 35400 32638 32617 32626]
label_truth:  32622
----------------------
Top-5 Label:  [32654 21556 33626 33617 26122]
label_truth:  32654
----------------------
Top-5 Label:  [32676 

Top-5 Label:  [37213 32792 20094 29486 29956]
label_truth:  36578
----------------------
Top-5 Label:  [22716 22737 34511 25972 40099]
label_truth:  36605
----------------------
Top-5 Label:  [32752 40138 28698 34649 39077]
label_truth:  36650
----------------------
Top-5 Label:  [32752 34643 36755 28698 29486]
label_truth:  36664
----------------------
Top-5 Label:  [36766 40505 31204 33300 39333]
label_truth:  36766
----------------------
Top-5 Label:  [38663 26216 29965 34107 33981]
label_truth:  36786
----------------------
Top-5 Label:  [36817 36828 36816 21280 36869]
label_truth:  36817
----------------------
Top-5 Label:  [36820 36825 36869 36816 36831]
label_truth:  36820
----------------------
Top-5 Label:  [35868 36855 36867 35820 36886]
label_truth:  36855
----------------------
Top-5 Label:  [36861 36834 36924 36843 36887]
label_truth:  36861
----------------------
Top-5 Label:  [36864 36861 28359 35887 35795]
label_truth:  36864
----------------------
Top-5 Label:  [31934 

In [36]:
def evaluate_style(imagepath):
    layer = 'icp3_in'
    
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_raw_scale('data', 255) 

    rightcount=0
    allcount=0

    gen_images = os.listdir(imagepath)
    gen_images = [img_name for img_name in gen_images if '_fake_B' in img_name]
    tar_images = os.listdir('../datasets/simhei_to_etl_0.2/testA')
    
    res = 0.0
    n = min(len(gen_images), len(tar_images))
    n = n if n%2==0 else n-1
    for i in range(0,n,1):
        res += np.linalg.norm(style_rep(os.path.join(imagepath, gen_images[i]),layer)-style_rep(os.path.join('../datasets/simhei_to_etl_0.2/testA/', tar_images[i]),layer))
    res /= n
    
    print ('Average style discrepancy %f'%res)

In [37]:
evaluate_style('/media/sinh/DATA/cinnamon/phase3/repos/pytorch-CycleGAN-and-pix2pix/results/exp_6_simhei_to_1252/test_latest/images')

Average style discrepancy 437.311653


In [38]:
res = 0.0
a_dir = '../datasets/simhei_to_etl_0.2/trainA'
a_dir_1 = '../datasets/simhei_to_etl_0.2/testA'
layer = 'icp3_in'
image_names = [os.path.join(a_dir,e) for e in os.listdir(a_dir)] + [os.path.join(a_dir_1, e) for e in os.listdir(a_dir_1)]
np.random.shuffle(image_names)
n = len(image_names)
n = n if n%2==0 else n-1
for i in range(0,n,2):
    res += np.linalg.norm(style_rep(image_names[i],layer)-style_rep(image_names[i+1],layer))
res /= n/2.0
print (res)

453.797321849


In [39]:
res = 0.0
a_dir = '../datasets/simhei_to_etl_0.2/trainA'
a_dir_1 = '../datasets/simhei_to_etl_0.2/testA'
b_dir = '../datasets/simhei_to_etl_0.2/trainB'
b_dir_1 = '../datasets/simhei_to_etl_0.2/testB'
layer = 'icp3_in'
image_a_names = [os.path.join(a_dir, e) for e in os.listdir(a_dir)] + [os.path.join(a_dir_1, e) for e in os.listdir(a_dir_1)]
np.random.shuffle(image_a_names)
image_b_names = [os.path.join(b_dir, e) for e in os.listdir(b_dir)] + [os.path.join(b_dir_1, e) for e in os.listdir(b_dir_1)]
np.random.shuffle(image_b_names)
n = min(len(image_a_names), len(image_b_names))
n = n if n%2==0 else n-1
for i in range(0,n,1):
    res += np.linalg.norm(style_rep(image_a_names[i],layer)-style_rep(image_b_names[i],layer))
res /= n
print (res)

499.893196112
